In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE
from sklearn.neural_network import MLPRegressor

In [3]:
d=2

In [8]:
n_components=200
def run_VAW(x_data, y_data):#передавать x_data надо с обрезкой до n
    np.random.seed(2)
    omega = np.random.normal(loc=0, scale=1, size=(n_components, d))
    b = np.random.uniform(-np.pi, np.pi, n_components)
    lam = 1
    X = np.hstack((np.cos(x_data@omega.T + b),np.sin(x_data@omega.T + b)))
    n = X.shape[0]

    A=np.zeros((X.shape[1],X.shape[1]))
    S=lam*np.eye(X.shape[1])
    Z=np.zeros(X.shape[1])
    A=X[:n][0].reshape(-1,1)@X[:n][0].reshape(1,-1)
    S+=A
    w=np.zeros((n, X.shape[1]))
    for t in range(n - 1):
        Z+=y_data[:n][t]*X[:n][t]
        A=X[:n][t+1].reshape(-1,1)@X[:n][t+1].reshape(1,-1)
        S+=A
        w[t+1]=np.linalg.solve(S, Z)
    w_hat=np.mean(w,axis=0)
    return w_hat, omega, b

In [6]:
def run_MLPRegressor(x_data, y_data):
    kr = MLPRegressor(warm_start = True)
    kr.fit(x_data, y_data)
    return kr

In [ ]:
# Loop through different values of n and s
for n in [10000, 50000, 100000]:
  for s in [5, 15]:
    print('n: ', n)
    print('s: ', s)
    d = 2
    np.random.seed(2)
    # Generate multivariate normal random variables
    eps = np.random.multivariate_normal(np.zeros(d), np.eye(d), n+s)
    kappa = 0.99
    beta = 0.1
    theta = 0.3
    # Rotation matrix
    Q = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    dt = 1
    B = np.sqrt(dt)*np.eye(d)
    x = np.zeros((n + s, d))
    y = np.zeros(n)

    # Define function g
    def g(x):
        return np.exp(-(beta*(np.dot(x,x)))/ 2)

    # Define function sigma
    def sigma(t):
        return np.sum([kappa**(2*j)*np.linalg.matrix_power(Q, j)@B@B.T@np.linalg.matrix_power(Q.T, j)for j in range(t)], axis= 0)

    # Define function h
    def h(x):
        return (1/np.sqrt(np.linalg.det(np.eye(d)+beta*sigma(s)))) * np.exp(-.5*kappa**(2*s)*beta*np.dot(np.linalg.inv(np.eye(d)+beta*sigma(s))@np.linalg.matrix_power(Q, s)@x, np.linalg.matrix_power(Q, s)@x))

    # Generate x values
    for t in range(1, n + s):
        x[t] = kappa*x[t-1] + eps[t]

    # Generate y values
    for t in range(n):
        y[t] = g(x[t + s])

    # Generate test data
    x_test = np.random.multivariate_normal(np.zeros(d), sigma(1000), 1000)
    x_test = np.sort(x_test, axis = 0)
    h_test = np.array([h(xi) for xi in x_test])
    z=x_test.reshape(-1,d)

    # Run MLP Regressor
    mlp = run_MLPRegressor(x[:n], y)
    mse_vaw = []
    # Run VAW multiple times and calculate MSE
    for i in range(3):
        VAW = run_VAW(x[:n], y)
        Z = np.hstack((np.cos(x_test@VAW[1].T + VAW[2]),np.sin(x_test@VAW[1].T + VAW[2])))
        mse_vaw.append(MSE(h_test, Z@VAW[0].T))
    # Print MSE results
    print('MLP MSE: ', MSE(h_test, mlp.predict(z)))
    print('VAW MSE: ', np.mean(mse_vaw))
    print('\n')

n:  10000
s:  5
MLP MSE:  0.005806721567182997
VAW MSE:  0.009465662640349034


n:  10000
s:  15
MLP MSE:  0.010790528824848343
VAW MSE:  0.01244729778097208


n:  50000
s:  5
